In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.stattools as ts

### STARGAZER ###

from stargazer.stargazer import Stargazer, LineLocation

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df_nzgas_excl_coal.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

df2 = df[(df['DATE'] >= '2009-3-1') & (df['DATE'] <= '2022-6-10')]

df3 = df[(df['DATE'] < '2009-3-1')]

print(df)

         DATE  Price_NZ    NZ_BTU  NZ_USD_Q  NZBTU_USD        HH    NBP_raw  \
0  1999-03-01  2.885704  3.044580  0.539400   1.642246  1.903333   9.825882   
1  1999-06-01  2.936804  3.098492  0.537400   1.665130  2.290000   9.510690   
2  1999-09-01  2.698549  2.847121  0.521433   1.484584  2.693333   9.767143   
3  1999-12-01  2.827617  2.983294  0.511567   1.526154  2.383333  10.796316   
4  2000-03-01  2.871244  3.029323  0.493033   1.493557  2.830000  14.691452   
..        ...       ...       ...       ...        ...       ...        ...   
86 2020-09-01  6.594163  6.957211  0.663333   4.614950  2.203333  23.882985   
87 2020-12-01  6.825113  7.200877  0.704467   5.072778  2.636667  36.814286   
88 2021-03-01  7.414286  7.822487  0.717100   5.609505  3.543333  49.338936   
89 2021-06-01  9.099814  9.600814  0.710567   6.822018  3.336667  60.539615   
90 2021-09-01  9.154261  9.658258  0.702267   6.782672  4.913333  54.578462   

     NBP_USD      Brent  GBP_USD  
0   1.605058  11

In [ ]:
fig = plt.gcf()
fig.set_size_inches(11.5, 6.5)
plt.plot(df['DATE'],df['HH'],color='blue')
plt.plot(df['DATE'],df['NZBTU_USD'],color='red')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Nominal USD', fontsize=12)
plt.legend(["Henry Hub Front Month", "New Zealand Wholesale Gas"], loc ="upper right")
plt.grid(color="black")

plt.savefig('/Users/joe/Dropbox/PhD/Writing/TeX/Charts/}nz-us-gas.png', dpi=400)

In [ ]:
result1 = smf.ols(formula="NZBTU_USD ~ HH", data=df).fit()
print(result1.summary())
plt.plot(result1.resid)

In [ ]:
test = ts.adfuller(result1.resid,autolag="AIC")
print(test)

result = ts.coint(df['HH'],df['NZBTU_USD'])
print(result)

In [ ]:
test1 = ts.adfuller(df2['HH'],autolag="AIC")
print(test1)
test2 = ts.adfuller(df2['NZBTU_USD'],autolag="AIC")
print(test2)


fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.plot(df2['DATE'],df2['HH'],color='blue')
plt.plot(df2['DATE'],df2['NZBTU_USD'],color='red')
plt.grid(color="black")

In [ ]:
result2 = smf.ols(formula="NZBTU_USD ~ HH", data=df2).fit()
print(result2.summary())
plt.plot(df2["DATE"],result2.resid)


test = ts.adfuller(result2.resid,autolag="AIC")
print(test)

result = ts.coint(df2['HH'],df2['NZBTU_USD'])
print(result)

In [ ]:
result_switch = smf.ols(formula="HH ~ NZBTU_USD", data=df2).fit()
print(result_switch.summary())
plt.plot(df2["DATE"],result_switch.resid)

test = ts.adfuller(result.resid,autolag="AIC")
print(test)

In [ ]:
test1 = ts.adfuller(df3['HH'],autolag="AIC")
print(test1)
test2 = ts.adfuller(df3['NZBTU_USD'],autolag="AIC")
print(test2)


fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.plot(df3['DATE'],df3['HH'],color='blue')
plt.plot(df3['DATE'],df3['NZBTU_USD'],color='red')
plt.grid(color="black")

In [ ]:
result3 = smf.ols(formula="NZBTU_USD ~ HH", data=df3).fit()
print(result3.summary())
plt.plot(df3["DATE"],result3.resid)

test = ts.adfuller(result3.resid,autolag="AIC")
print(test)

In [ ]:
df['ratio'] = df["HH"]/df["NZBTU_USD"]
plt.plot(df['DATE'],df['ratio'],color='blue')

In [ ]:
mod = sm.tsa.MarkovRegression(df['ratio'].iloc[1:], k_regimes=2, order=1, switching_variance=True)
res_mod = mod.fit()
res_mod.summary()

In [ ]:
res_mod.filtered_marginal_probabilities[1].plot(
    title='Probability of being in high price/volatility regime', figsize=(12,2))

In [ ]:
fig = plt.gcf()
fig.set_size_inches(10.5, 8)
plt.plot(df['DATE'],df['NZ_BTU'],color='green')
fig.suptitle('Quarterly New Zealand Natural Gas Prices',fontsize=18)
plt.xlabel('Date', fontsize=12)
plt.ylabel('NZD/BTU', fontsize=12)
plt.grid(color="black")

plt.savefig('C:/Users/User/Dropbox/PhD/Writing/TeX/Charts/nzbtu.png', dpi=400)

In [ ]:
star = Stargazer([result1,result2,result3])
print(star.render_latex())

In [ ]:
fig = plt.gcf()
fig.set_size_inches(11.5, 6.5)
plt.plot(df['DATE'],df['NBP_USD'],color='blue')
plt.plot(df['DATE'],df['NZBTU_USD'],color='red')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Nominal USD', fontsize=12)
plt.legend(["NBP Front Month", "New Zealand Wholesale Gas"], loc ="upper right")
plt.grid(color="black")

plt.savefig('/Users/joe/Dropbox/PhD/Writing/TeX/Charts/}nz-uk-gas.png', dpi=400)

In [ ]:
result_nbp = smf.ols(formula="NZBTU_USD ~ NBP_USD", data=df).fit()
print(result_nbp.summary())
plt.plot(result_nbp.resid)
test = ts.adfuller(result_nbp.resid,autolag="AIC")
print(test)

In [ ]:
result = ts.coint(df['NZ_BTU'],df['NBP_USD'])
print(result)

In [ ]:
star = Stargazer([result_nbp])
print(star.render_latex())

In [ ]:
result2_nbp = smf.ols(formula="NZBTU_USD ~ NBP_USD", data=df2).fit()
plt.plot(result2_nbp.resid)
test2 = ts.adfuller(result2_nbp.resid,autolag="AIC")
print(test2)

In [ ]:
star = Stargazer([result_nbp, result2_nbp, result3_nbp ])
print(star.render_latex())

In [ ]:
result3_nbp = smf.ols(formula="NZBTU_USD ~ NBP_USD", data=df3).fit()
plt.plot(result3_nbp.resid)
test3 = ts.adfuller(result3_nbp.resid,autolag="AIC")
print(test3)

In [ ]:
result4 = smf.ols(formula="NZBTU_USD ~ NBP_USD + Brent + HH", data=df3).fit()
plt.plot(result4.resid)
test4 = ts.adfuller(result4.resid,autolag="AIC")
print(test4)

In [ ]:
print(result4.summary())

In [17]:
df['NCF'].describe()

KeyError: 'NCF'

In [27]:
test = ts.adfuller(df['HH'],autolag="AIC")
print(test)

(-3.2042838048935542, 0.019738050976324466, 1, 89, {'1%': -3.506057133647011, '5%': -2.8946066061911946, '10%': -2.5844100201994697}, 231.75939066334746)
